# Products table creation

In [79]:
import pandas as pd
import numpy as np

In [80]:
products = pd.read_excel("original_data/products.xlsx")

In [81]:
# Translate some of the variables:
products["type"].replace({"Isotonic": "isotonic", "Pastilles": "pills", "Gel": "gel"}, inplace=True)

products["tendency"].replace({"mitja": "strandard", "alta": "high", "baixa": "low", "bixa": "low"}, inplace=True)

products.drop("RESTRICCIONS DOSI", axis=1, inplace=True)

products["vegan"].fillna(False, inplace=True)
products["vegan"].replace({"Sí": True}, inplace=True)

products["gluten_free"].fillna(False, inplace=True)
products["gluten_free"].replace({"Sí": True, "Sí ": True}, inplace=True)
products["gluten"] = products["gluten_free"].replace({True: False, False: True})

products["lactose"].fillna(True, inplace=True)
products["lactose"].replace({"-": True, "No conté": False, " Pot contenir tz": True}, inplace=True)

products["egg_protein"].fillna(True, inplace=True)
products["egg_protein"].replace({"-": True, "No conté": False, " Pot contenir tz": True}, inplace=True)

products["nuts"].fillna(True, inplace=True)
products["nuts"].replace({"-": True, "No conté": False, " Pot contenir tz": True}, inplace=True)

products["soy"].fillna(True, inplace=True)
products["soy"].replace({"-": True, "No conté": False, " Pot contenir tz": True}, inplace=True)

products["natural"].fillna(False, inplace=True)
products["natural"].replace({"Sí ": True}, inplace=True)

In [82]:
# The salts must be in grams:
products["g_salts"] = products["mg_salts"] / 1000

# Tests Kilian Garcia Salts calculations

In [83]:
# Volatges from the 3 trainings:
voltages = {"voltage1": 0.826, "voltage2": 0.818, "voltage3": 0.803}

# Initial data from the app:
training_asfalto1 = {"trainingid": "t1", "initial_weight": 64.4, "final_weight": 63, "water_intake": 0.25, "time": 1.15, "total_distance": 16.74, "total_elevation": 0, "times_orinated": 0}
training_asfalto2 = {"trainingid": "t2", "initial_weight": 66.2, "final_weight": 64.5, "water_intake": 1.5, "time": 2.066, "total_distance": 30, "total_elevation": 0, "times_orinated": 0}

training_mountain1 = {"trainingid": "t3", "initial_weight": 65.8, "final_weight": 64, "water_intake": 1.25, "time": 2.033, "total_distance": 25.4, "total_elevation": 0, "times_orinated": 0}
training_mountain2 = {"trainingid": "t4", "initial_weight": 66.4, "final_weight": 64.5, "water_intake": 1, "time": 1.88, "total_distance": 20.85, "total_elevation": 706, "times_orinated": 0}

# Create the dataframe:
data = pd.DataFrame([training_asfalto1, training_asfalto2, training_mountain1, training_mountain2])

In [84]:
def calculate_salts_concentration(voltage: float) -> float:
    return (voltage - 0.42141) / (0.00819)

# Calculate the concentration of salts:
concentration_of_salts = {
    "concentration_1": calculate_salts_concentration(voltages["voltage1"]),
    "concentration_2": calculate_salts_concentration(voltages["voltage2"]),
    "concentration_3": calculate_salts_concentration(voltages["voltage3"]),
}

# Calculate the mean:
concentration_of_salts_mean = sum(concentration_of_salts.values()) / len(concentration_of_salts)
# This mean will be added as if it is the concentration of salts of each one of the trainings:
data["concentration_of_salts"] = concentration_of_salts_mean

# Calculate weight loss:
data["weight_loss"] = data["initial_weight"] - data["final_weight"] + data["water_intake"] - data["times_orinated"]*0.25

# Define the function to calculate lost salts:
def calculate_lost_salts(row):
    return (row["concentration_of_salts"] * 58.44 * 0.9 * row["weight_loss"]) / 1000

# Calculate lost salts:
data["lost_salts"] = data.apply(calculate_lost_salts, axis=1)

# Calculate the intensity:
data["avg_intensity"] = data["total_distance"] / data["time"]

In [85]:
# Separate the data:
asfalt_data = data.loc[(data["trainingid"].isin(["t1", "t2"]))]
mountain_data = data.loc[(data["trainingid"].isin(["t3", "t4"]))]

In [86]:
# Objectives for the two races:
asfalt_objectives = {"distance": 42, "time": 2.5}
mountain_objectives = {"distance": 25, "time": 2}

# Calculate the objective velocity:
asfalt_objectives["avg_intensity"] = asfalt_objectives["distance"] / asfalt_objectives["time"]
mountain_objectives["avg_intensity"] = mountain_objectives["distance"] / mountain_objectives["time"]

In [87]:
from sklearn.linear_model import LinearRegression
model_asfalt = LinearRegression()

# Fit the regression:
model_asfalt.fit(asfalt_data[["time"]].values, asfalt_data["lost_salts"])

# Predict the lost of salts:
loss_salts_prediction_asfalt_ = model_asfalt.predict(np.array([[asfalt_objectives["time"]]]))

# Do the same for the mountain:
model_mountain = LinearRegression()
model_mountain.fit(mountain_data[["time"]], mountain_data["lost_salts"])
loss_salts_prediction_mountain_ = model_mountain.predict(np.array([[mountain_objectives["time"]]]))

/home/leibniz/anaconda3/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [88]:
print(loss_salts_prediction_asfalt_)
print(loss_salts_prediction_mountain_)

[9.96150871]
[7.64040367]


# Hydration plan generation asfalt

## Product 1

In [106]:
# Convert it to a float since now it is an array:
loss_salts_prediction_asfalt = loss_salts_prediction_asfalt_[0]
loss_salts_prediction_mountain = loss_salts_prediction_mountain_[0]

# Subtract some of the salts because the athlete loses a lot:
loss_salts_prediction_asfalt = loss_salts_prediction_asfalt
loss_salts_prediction_mountain = loss_salts_prediction_mountain

brand_series = pd.concat([products["brand"], pd.Series([None, None, None])], ignore_index=True)

choosen_brand = brand_series.sample(n=1).iloc[0]

choosen_format = pd.Series(["pills", "gel", None]).sample(n=1).iloc[0]

user_conditions = pd.Series(["vegan", "gluten", "lactose", "egg_protein", "nuts", "soy"]).sample(n=2).to_list()

In [107]:
print(loss_salts_prediction_mountain)
print(loss_salts_prediction_asfalt)

print(choosen_brand)
print(choosen_format)

7.640403668261149
9.961508705125013
Santa Madre
pills


In [108]:
class HydrationPlanGenerator():

    def __init__(self, products: pd.DataFrame, loss_salts_prediction: float, choosen_brand: str, choosen_format: str, obj_time: float):
        self.products = products
        self.choosen_brand = choosen_brand
        self.choosen_format = choosen_format
        self.obj_time = obj_time

        self.vspre = loss_salts_prediction * 0.25
        self.vsduring = loss_salts_prediction * 0.5
        self.vspost =  loss_salts_prediction * 0.25

        self.isotonics = self.products[self.products["type"] == "isotonic"]
        self.gels = self.products[self.products["type"] == "gel"]
        self.pills = self.products[self.products["type"] == "pills"]

        self.product1 = None
        self.product2 = None
        self.litters_product3 = None
        self.product4 = None
        self.product5 = None

    def get_brand_product(self, products: pd.DataFrame) -> pd.Series:
        """
        The function returns the product of a certain type of the choosen brand

        Used for product1 and product2
        """

        # This function should be an SQL call once we have the DB
        brand_product = products.loc[products["brand"] == self.choosen_brand]

        # Sort the products from most salts to less salts:
        brand_product = brand_product.sort_values(by="g_salts", ascending=False)

        # Return all the products from the brand:
        return brand_product

    def find_max_isotonic(self) -> pd.Series:
        """
        Finds the isotonic with the highest concentration of salts
        """

        return self.isotonics.loc[self.isotonics["g_salts"] == max(self.isotonics["g_salts"])].iloc[0]

    def find_isotonics_with_enough_salts(self) -> pd.DataFrame:
        """
        Finds the isotonics with enough salts
        """
        # Get the isotonics that satisfy the requirements of salts of the user:

        mask = (self.vspre - 1) <= (self.isotonics["g_salts"] / 2)

        isotonics_with_enough_salts = self.isotonics.loc[mask]

        # Sort the values in descendent order by the grams of salts they contain:
        isotonics_with_enough_salts = isotonics_with_enough_salts.sort_values(by="g_salts")

        return isotonics_with_enough_salts

    def isotonic_enough_salts(self, isotonic: pd.Series, factor: float) -> bool:
        """
        Checks if the isotonic satisfies the salts requirements depending on the factor
        """
        enough_salts = (self.vspre - factor) <= (isotonic["g_salts"] / 2)

        # For clarity of the code we do not return the bool enough_salts:
        if enough_salts:
            return True
        else:
            return False

    def product1_recommendation_fallback(self) -> pd.Series:
        """
        Funtion that finds the isotonic in case that:
            - The user has no choosen brand
            - The brand of the user has no isotonic
            - The user has a choosen brand but the isotonic of the brand has not enough salts
        """

        # Get all the isotonics that satisfy the salts requirements in order from less salts to most salts:
        isotonics = self.find_isotonics_with_enough_salts()

        # If there are no isotonics that satisfy the salt requirements:
        if isotonics.empty:
            print("The salts requirements are not satisfied. Recommending the maximum isotonic")
            # Return the isotonic with the maximum number of salts.
            return self.find_max_isotonic()
        
        # If there are isotonics that satisfy the rule 
        else:
            print("There are isotonics that satisfy the salts requirements. Recommending the one with less salts")
            # return the isotonic with less salts that satisfies the salts requirements. This isotonic is in the first row of the dataframe isotonics:
            return isotonics.iloc[0]

    def get_product1(self) -> str:
        """
        Returns the recomendation for the product 1
        """
        print("Product 1:")

        # if the user has choosen a brand:
        if self.choosen_brand:
            print("The user has a prefered brand")
            # Get all the isotonics from the brand:
            brand_isotonic = self.get_brand_product(self.isotonics)

            # If choosen brand does not have an isotonic:
            if brand_isotonic.empty:
                print("The prefered brand does not have an isotonic")
                return self.product1_recommendation_fallback()

            else:
                print("The prefered brand has an isotonic")

                # Since the brand can have multiple isotonics we will get the one with the highest quantity of
                # salts. This product is going to be located in the first row of the dataframe brand_isotonic
                # since the funtion get_brand_product() returns the isotonics ordered by most salts to less salts:
                brand_isotonic = brand_isotonic.iloc[0]

                # if the isotonic from the choosen brand has enough salts:
                if self.isotonic_enough_salts(brand_isotonic, factor=1):
                    print("The isotonic from the prefered brand has enough salts")
                    return brand_isotonic
                
                # If the isotonic from the choosen brand has not enough salts:
                else:
                    print("The isotonic from the prefered brand does not have enough salts")
                    return self.product1_recommendation_fallback()

        # If the user has not choosen a brand:
        else:
            print("The user does not have a prefered brand")
            return self.product1_recommendation_fallback()
        
    def find_product2(self, products: pd.DataFrame):
        """
        Input: Dataframe products containing all the pills or all the
        gels depending on the choice of the user

        Finds the product 2 in case that product 1 is not enough
        """
        
        # Check if the user has choosen a brand:
        if self.choosen_brand:
            print("User has choosen a brand")
            # Get all the gels/ pills from the choosen brand:
            brand_product = self.get_brand_product(products)

            # If the choosen brand has no gel/pills
            if brand_product.empty:
                print("The choosen brand has no gel/pills")
                return self.product2_recommendation_fallback(products)
            
            # If the choosen brand has gel/pills:
            else:
                print("The choosen brand has gel/pills")

                # Since the brand can have multiple gels/pills we will get the one with the highest quantity of
                # salts. This product is going to be located in the first row of the dataframe brand_product
                # since the funtion get_brand_product() returns the gels/pills ordered by most salts to less salts:
                brand_product = brand_product.iloc[0]
                
                # If the brand product has enough salts:
                if self.product2_enough_salts(brand_product):
                    print("The gel/pill of the choosen brand has enough salts. Recommending it...")
                    return brand_product

                # If the brand product does not have enough salts:
                else:
                    print("The gel/pill of the choosen brand does not have enough salts")
                    return self.product2_recommendation_fallback(products)
        
        # if the user has not choosen a brand:
        else:
            print("User has not choosen a brand")
            return self.product2_recommendation_fallback(products)
        
    def find_product2_with_enough_salts(self, products: pd.DataFrame):
        """
        Finds the pills/gels with enough salts
        """
        # Get the pills/gels that satisfy the requirements of salts of the user:

        mask = (products["g_salts"] >= (self.vspre - self.product1["g_salts"] / 2))

        products_with_enough_salts = products.loc[mask]

        # Sort the values in descendent order by the grams of salts they contain:
        products_with_enough_salts = products_with_enough_salts.sort_values(by="g_salts")

        return products_with_enough_salts

    def find_max_product2(self, products: pd.DataFrame):
        """
        Finds the pill/gel with the highest concentration of salts
        """

        return products.loc[products["g_salts"] == max(products["g_salts"])].iloc[0]

    def product2_recommendation_fallback(self, products: pd.DataFrame):
        """
        Input: Dataframe products containg all the pills or all the
        gels depending on the choice of the user

        Funtion that finds the gel/pill recommendation in case that:
            - The user has not choosen a brand
            - The brand that the user has choosen has no gels/pills
            - The gel/pill from the choosen brand does not satisfy the salt requirements of the user
        """

        # Get all the pills/gels that satisfy the salts requirements in order from less salts to most salts:
        products_enough_salts = self.find_product2_with_enough_salts(products)

        # If there are no isotonics that satisfy the salt requirements:
        if products_enough_salts.empty:
            print("The salts requirements are not satisfied. Recommending the maximum pill/gel")
            # Return the pill/gel with the maximum number of salts.
            return self.find_max_product2(products)
        
        # If there are isotonics that satisfy the rule 
        else:
            print("There are pills/gels that satisfy the salts requirements. Recommending the one with less salts")
            # return the isotonic with less salts that satisfies the salts requirements. This isotonic is in the first row of the dataframe isotonics:
            return products_enough_salts.iloc[0]


    def product2_enough_salts(self, brand_product: pd.Series) -> bool:
        """
        Checks if the gel/pill has enough salts in relation with the product1 isotonic
        """

        enough_salts = (brand_product["g_salts"] >= (self.vspre - self.product1["g_salts"] / 2))

        # Do not return enough_salts directly for code clarity:
        if enough_salts:
            return True
        else:
            return False
        
    def get_product2(self) -> pd.Series:
        """
        Returns the recommendation for the product 2
        """
        print("Product 2:")

        # First we have to check if the isotonic from product 1 satisfies the salt requirements:
        if self.isotonic_enough_salts(self.product1, factor=0.4):
            print("No need for product 2")
            # There is no need to recommend product 2:
            return None
        
        else:
            # In case that the salts requirements are not fully satisfied with product 1:
            print("There is need for product 2")
            print(self.choosen_format)

            if ((self.choosen_format is None) or (self.choosen_format == "gel")):
                print(f"The user has choosen the format {self.choosen_format}")
                
                return self.find_product2(self.gels)

            if self.choosen_format == "pills":
                print(f"The user has choosen the format {self.choosen_format}")

                return self.find_product2(self.pills)
            
    def get_litters_product3(self) -> pd.Series:
        """
        Returns the amount of litters for product 3. Product 1 and Product 3 are the same product
        """
        return int(self.obj_time) * 0.75
    
    def get_units_product4(self) -> pd.Series:
        """
        Returns the units of product 4
        """

        units = (self.vsduring - self.product1["g_salts"] * self.litters_product3) / (self.product2["g_salts"])

        if units <= 0:
            return 0
        else:
            # We have to round the decimal number up:
            return int(units) + 1
        
        # TODO: We have to check if we are passing the restrictions of the recommended product

    def get_litters_product5(self) -> pd.Series:
        """
        Returns the litters for product 5
        """
        # We have to check if the amount of salts of product 1 are enough to recommend 0,5 litters
        # or if we have to recommend 1 litter
        enough_salts = (self.product1["g_salts"] / 2) >= self.vspost

        if enough_salts:
            return 0.5
        else:
            return 1
    
    def create_plan(self):
        """
        Calls all the get_product functions of the class to 
        """
        self.product1 = self.get_product1()
        self.product2 = self.get_product2()
        self.litters_product3 = self.get_litters_product3()
        self.units_product4 = self.get_units_product4()
        self.litters_product5 = self.get_litters_product5()

In [112]:
hydration_plan_generator = HydrationPlanGenerator(
    products = products,
    loss_salts_prediction = loss_salts_prediction_mountain,
    choosen_brand = choosen_brand,
    choosen_format = choosen_format,
    obj_time = mountain_objectives["time"]
)

In [113]:
hydration_plan_generator.create_plan()

Product 1:
The user has a prefered brand
The prefered brand has an isotonic
The isotonic from the prefered brand does not have enough salts
There are isotonics that satisfy the salts requirements. Recommending the one with less salts
Product 2:
There is need for product 2
pills
The user has choosen the format pills
User has choosen a brand
The choosen brand has no gel/pills
There are pills/gels that satisfy the salts requirements. Recommending the one with less salts


In [114]:
print("Product 1:")
print(f"{hydration_plan_generator.product1['name']}, brand: {hydration_plan_generator.product1['brand']}")
print("Product 2:")
print(f"{hydration_plan_generator.product2['name']}, brand: {hydration_plan_generator.product2['brand']}")
print("Product 3 litters:")
print(hydration_plan_generator.litters_product3)
print("Product 4 units:")
print(hydration_plan_generator.units_product4)
print("Product 5 litters")
print(hydration_plan_generator.litters_product5)

Product 1:
Low-calorie electrolyte powder for sports & exercise, brand: Huma
Product 2:
PRO SALT CAPS, brand: Crown
Product 3 litters:
1.5
Product 4 units:
1
Product 5 litters
1


## Comprovacions

In [18]:
hydration_plan_generator.product2

type                                                                     gel
brand                                                            Santa Madre
name                                       Unusual gel - on off caf - 45 cho
link                       https://santamadreco.com/es/shop/57-246-gel-en...
mg_salts                                                                 500
tendency                                                                 low
daily_limit                                                              NaN
grams_per_litter                                                         NaN
minimum_dosage_interval                                                  1.0
vegan                                                                   True
gluten_free                                                             True
lactose                                                                False
egg_protein                                                            False

In [238]:
hydration_plan_generator.choosen_brand

'Totum'

In [51]:
int(2.1)

2

In [242]:
isotonics = products.loc[(products["type"] == "isotonic")]

In [243]:
mask = ((hydration_plan_generator.vspre - 0.65) <= isotonics["g_salts"]/2)
candidates = isotonics.loc[mask]
candidates

,type,brand,name,link,mg_salts,tendency,daily_limit,grams_per_litter,minimum_dosage_interval,vegan,gluten_free,lactose,egg_protein,nuts,soy,natural,gluten,g_salts
1,isotonic,Nutrinovex,Hidratein sals,https://nutrinovex.es/hidratein-salts/,3020,high,NaN,100.0,NaN,False,False,True,True,True,True,False,True,3.02


In [244]:
isotonics = products.loc[(products["type"] == "isotonic")]
mask = ((hydration_plan_generator.vspre - 0.65) <= isotonics["g_salts"]/2)

product1 = isotonics.loc[mask].iloc[0]

In [245]:
product1

type                                                     isotonic
brand                                                  Nutrinovex
name                                               Hidratein sals
link                       https://nutrinovex.es/hidratein-salts/
mg_salts                                                     3020
tendency                                                     high
daily_limit                                                   NaN
grams_per_litter                                            100.0
minimum_dosage_interval                                       NaN
vegan                                                       False
gluten_free                                                 False
lactose                                                      True
egg_protein                                                  True
nuts                                                         True
soy                                                          True
natural   

In [246]:
product1["g_salts"] / 2 >= hydration_plan_generator.vspre - 0.4

False

In [247]:
print(hydration_plan_generator.choosen_format)
print(hydration_plan_generator.choosen_brand)

pills
Totum


In [250]:
candidate = products[(products["brand"] == "Totum") & (products["type"] == "pills")]

In [251]:
candidate

,type,brand,name,link,mg_salts,tendency,daily_limit,grams_per_litter,minimum_dosage_interval,vegan,gluten_free,lactose,egg_protein,nuts,soy,natural,gluten,g_salts


In [216]:
candidate["g_salts"] >= hydration_plan_generator.vspre - hydration_plan_generator.product1["g_salts"] / 2

8    False
Name: g_salts, dtype: bool

In [255]:
mask = (products["g_salts"] >= (hydration_plan_generator.vspre - hydration_plan_generator.product1["g_salts"]/2))
pills = products.loc[mask & (products["type"] == "pills")]

In [256]:
pills

,type,brand,name,link,mg_salts,tendency,daily_limit,grams_per_litter,minimum_dosage_interval,vegan,gluten_free,lactose,egg_protein,nuts,soy,natural,gluten,g_salts
14,pills,Nutrinovex,Hidratein Salts Caps – Duplo,https://nutrinovex.es/hidratein-salts-caps-duplo/,515,strandard,NaN,NaN,0.75,True,True,True,True,True,True,False,False,0.515
15,pills,226ers,SALTS ELECTROLYTES,https://www.226ers.com/es/salts-electrolytes-3...,474,low,14.0,NaN,0.50,True,True,False,False,False,True,False,False,0.474
16,pills,226ers,SUB9 SALTS ELECTROLYTES,https://www.226ers.com/es/sub9-salts-electroly...,699,high,8.0,NaN,0.50,True,True,False,False,False,True,False,False,0.699
17,pills,Crown,PRO SALT CAPS,https://crownsportnutrition.com/producto/pro-s...,843,high,NaN,NaN,0.33,True,True,False,False,False,False,False,False,0.843
18,pills,Hammer,Endurolytes extreme,https://www.hammernutrition.eu/es/shop/endurol...,415,low,NaN,NaN,NaN,True,False,True,True,True,True,True,True,0.415
19,pills,Salt Sticks,Electrolyte caps,https://trideporte.com/suplementos/4636-82724-...,633,strandard,NaN,NaN,0.50,True,True,True,True,True,True,True,False,0.633


In [257]:
hydration_plan_generator.obj_time * 0.75

1.5

In [258]:
hydration_plan_generator.vsduring

3.8202018341305743

In [259]:
hydration_plan_generator.product1["g_salts"] * 1.5

4.53

In [260]:
hydration_plan_generator.product1["g_salts"] / 2 >= hydration_plan_generator.vspost

False

In [29]:
hydration_plan_generator.vsduring - hydration_plan_generator.product1["g_salts"] * 0.75

1.5552018341305742

In [23]:
hydration_plan_generator.vspre

1.9101009170652872

In [28]:
hydration_plan_generator.litters_product3

1.5

In [22]:
hydration_plan_generator.litters_product3

1.5

# intoleràncies

In [112]:
products.loc[(products["type"] == "isotonic") & (products["nuts"] == False)]

,type,brand,name,link,mg_salts,tendency,daily_limit,grams_per_litter,minimum_dosage_interval,vegan,gluten_free,lactose,egg_protein,nuts,soy,natural,gluten,g_salts
2,isotonic,Crown,BEBIDA ISOTÓNICA (ISODRINK & ENERGY),https://crownsportnutrition.com/producto/isodr...,1500,strandard,NaN,80.0,NaN,True,True,False,False,False,False,False,False,1.5


In [110]:
hydration_plan_generator.vspre - 0.65 <= ex["g_salts"] / 2

False